this code generates the descriptors from the dataframe. And exports as a csv.

In [2]:
#import all the modeules we've used before

import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import rdkit
# Render the figure in a notebook:
%matplotlib inline  

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score

In [3]:
df = pd.read_csv('toxicity_no_toxic_nans.csv')
#make two toxicity scores based on NR and SR assay response.
# df['NR_tox_score'] = df['NR-AR']+df['NR-AR-LBD']+df['NR-AhR']+df['NR-Aromatase']+df['NR-ER']+df['NR-ER-LBD']+df['NR-PPAR-gamma']
# df['SR_tox_score'] = df['SR-ARE']+df['SR-ATAD5']+df['SR-HSE']+df['SR-MMP']+df['SR-p53']
# df_dropped_Al = df.drop([1322, 2290,2297,3558, 4565,4649, 5538, 6723, 7830]) #we dropped these values, they're no longer there
df

,Unnamed: 0,smiles,mol_id,toxic
0,0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,TOX3021,1
1,4,CC(O)(P(=O)(O)O)P(=O)(O)O,TOX20800,0
2,6,O=S(=O)(Cl)c1ccccc1,TOX6619,0
3,7,O=C(O)Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1,TOX25232,1
4,12,CC(C)COC(=O)C(C)C,TOX6612,0
...,...,...,...,...
5270,7818,c1cnc(N2CCN(Cc3ccc4c(c3)OCO4)CC2)nc1,TOX25188,0
5271,7827,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,TOX2370,1
5272,7828,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,TOX2371,1
5273,7829,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...,TOX2377,1


In [4]:
# https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html
from rdkit.Chem import Descriptors
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule
            missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [5]:
desc_list = []
for molecule in range(len(df)):
    desc_list.append(getMolDescriptors(Chem.MolFromSmiles(df.iloc[molecule]['smiles'])))


[13:53:07] Explicit valence for atom # 4 Al, 6, is greater than permitted
Traceback (most recent call last):
  File "C:\Users\dx21006\AppData\Local\Temp\ipykernel_16092\1571356098.py", line 11, in getMolDescriptors
    val = fn(mol)
  File "c:\Users\dx21006\.conda\envs\chem-ml-env\Lib\site-packages\rdkit\Chem\EState\EState.py", line 91, in MaxAbsEStateIndex
    return max(abs(x) for x in EStateIndices(mol, force))
                               ~~~~~~~~~~~~~^^^^^^^^^^^^
  File "c:\Users\dx21006\.conda\envs\chem-ml-env\Lib\site-packages\rdkit\Chem\EState\EState.py", line 47, in EStateIndices
    nAtoms = mol.GetNumAtoms()
             ^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent call last):
  File "C:\Users\dx21006\AppData\Local\Temp\ipykernel_16092\1571356098.py", line 11, in getMolDescriptors
    val = fn(mol)
  File "c:\Users\dx21006\.conda\envs\chem-ml-env\Lib\site-packages\rdkit\Chem\EState\EState.py", line 77, in MaxEState

In [6]:
df_desc = pd.DataFrame(desc_list)
df_desc
#this is now a list containing all the descriptors

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,11.107593,11.107593,0.073011,-3.723219,0.900283,11.875000,258.324,248.244,258.013284,86.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,10.251875,10.251875,0.383488,-5.197724,0.365583,15.090909,206.027,197.963,205.974526,68.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.598302,10.598302,0.135802,-3.530602,0.609724,11.300000,176.624,171.584,175.969878,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.775986,10.775986,0.010525,-0.858003,0.489304,10.428571,621.934,612.862,621.763503,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.825046,10.825046,0.001157,-0.103056,0.564836,10.600000,144.214,128.086,144.115030,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,5.437662,5.437662,0.327913,0.327913,0.856312,17.727273,298.346,280.202,298.142976,114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5271,12.146718,12.146718,0.245103,0.245103,0.701319,49.000000,314.469,284.229,314.224580,126.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5272,11.787348,11.787348,0.082288,-0.082288,0.734230,52.714286,288.431,260.207,288.208930,116.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5273,10.372487,10.372487,0.088273,-0.088273,0.757170,42.700000,272.388,248.196,272.177630,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#recombine so this now contains SMILES, and ID.
df_desc['smiles'] = df['smiles']
df_desc['mol_id'] = df['mol_id']
df_desc['toxic'] = df['toxic']
df_desc

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,smiles,mol_id,toxic
0,11.107593,11.107593,0.073011,-3.723219,0.900283,11.875000,258.324,248.244,258.013284,86.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,TOX3021,1
1,10.251875,10.251875,0.383488,-5.197724,0.365583,15.090909,206.027,197.963,205.974526,68.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC(O)(P(=O)(O)O)P(=O)(O)O,TOX20800,0
2,10.598302,10.598302,0.135802,-3.530602,0.609724,11.300000,176.624,171.584,175.969878,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O=S(=O)(Cl)c1ccccc1,TOX6619,0
3,10.775986,10.775986,0.010525,-0.858003,0.489304,10.428571,621.934,612.862,621.763503,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O=C(O)Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1,TOX25232,1
4,10.825046,10.825046,0.001157,-0.103056,0.564836,10.600000,144.214,128.086,144.115030,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC(C)COC(=O)C(C)C,TOX6612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,5.437662,5.437662,0.327913,0.327913,0.856312,17.727273,298.346,280.202,298.142976,114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c1cnc(N2CCN(Cc3ccc4c(c3)OCO4)CC2)nc1,TOX25188,0
5271,12.146718,12.146718,0.245103,0.245103,0.701319,49.000000,314.469,284.229,314.224580,126.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,TOX2370,1
5272,11.787348,11.787348,0.082288,-0.082288,0.734230,52.714286,288.431,260.207,288.208930,116.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,TOX2371,1
5273,10.372487,10.372487,0.088273,-0.088273,0.757170,42.700000,272.388,248.196,272.177630,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...,TOX2377,1


In [ ]:
df_desc.to_csv('no_nontoxic_nans_desc_and_toxicity.csv')